In [48]:
import pandas as pd
from typing import Any, cast
import json

In [49]:
data_folder = "./json"
outcome_file = 'outcomes.json'
data_files = [
    "action.json",
    "attack.json",
    "magic.json",
    "shotlock.json",
]

In [50]:
def load_json(file:str):
    loaded_dict = {}
    with open(f'{data_folder}/{file}', 'r') as f:
        loaded_dict  = json.load(f)
    return loaded_dict

def create_outcomes_df(outcomes_dict:dict[str, str]) -> pd.DataFrame:
    crystals = cast(list[str], outcomes_dict.get('Crystals', []))
    outcomes_dict.pop('Crystals')
    outcomes_df = pd.DataFrame.from_dict(outcomes_dict, columns=crystals, orient='index')
    return outcomes_df


outcomes_dict: dict[str,str] = load_json(outcome_file)
outcome_df = create_outcomes_df(outcomes_dict)
print(outcome_df.head())


       Shimmering      Fleeting       Pulsing      Wellspring       Soothing  \
A      Fire Boost   Magic Haste   Leaf Bracer  Air Combo Plus       HP Boost   
B      Fire Boost  Reload Boost  Finish Boost       Once More  Damage Syphon   
C     Fire Screen  Attack Haste  Finish Boost      Combo Plus       HP Boost   
D     Fire Screen  Attack Haste   Leaf Bracer      Combo Plus       HP Boost   
E  Blizzard Boost   Magic Haste   Leaf Bracer      Combo Plus     Item Boost   

          Hungry        Abounding  
A  HP Prize Plus  Link Prize Plus  
B  HP Prize Plus       EXP Chance  
C  HP Prize Plus  Link Prize Plus  
D  HP Prize Plus  Link Prize Plus  
E  HP Prize Plus     Lucky Strike  


In [51]:
def create_recipe_df(recipe_list: list[dict[str,str]], columns) -> pd.DataFrame:
    recipe_df = pd.DataFrame.from_records(recipe_list)
    recipe_df.columns = columns

    characters = ['terra', 'ventus', 'aqua']


    def filter_characters(x:list[str], char: str) -> bool:
        return char in x

    for character in characters:
        recipe_df[character] = recipe_df['Used By'].apply(filter_characters, char=character)

    recipe_df.drop('Used By', axis=1, inplace=True)
    
    
    return recipe_df

def create_recipe_dfs():
    recipe_dfs: list[tuple[str, pd.DataFrame]] = []
    for file in data_files:
        filename = file.split('.')[0]

        data_dict: dict[str, Any] = load_json(file)

        header: dict[str,str] | None = data_dict.get('header')
        if not header:
            raise ValueError(f'No header found in {file}')
        
        columns = list(header.values())

        recipes: list[dict[str,str]] | None = data_dict.get('recipes')
        if not recipes:
            raise ValueError(f'No recipes found in {file}')
        
        recipe_df = create_recipe_df(recipes, columns)
        recipe_dfs.append((filename,recipe_df))
    return recipe_dfs

recipe_dfs = create_recipe_dfs()

for name, df in recipe_dfs:
    print(name)
    print(df.head())
    
    

action
        Command 1st Ingredient 2nd Ingredient Type    %  terra  ventus   aqua
0  Homing Slide   Sliding Dash        Magnera    P  100  False    True   True
1  Homing Slide   Sliding Dash      Air Slide    C  100  False    True   True
2  Homing Slide         Magnet      Air Slide    L  100  False    True   True
3  Thunder Roll  Thunder Surge     Dodge Roll    P   80  False    True  False
4  Thunder Roll       Thundaga     Dodge Roll    L   80  False    True  False
attack
        Command 1st Ingredient 2nd Ingredient Type    %  terra  ventus   aqua
0         Blitz    Quick Blitz      Slot Edge    O   90   True    True   True
1         Blitz  Barrier Surge   Wishing Edge    P   90  False   False   True
2         Blitz      Stun Edge      Slot Edge    K   90   True    True   True
3  Meteor Crash          Blitz          Quake    N  100   True   False  False
4  Meteor Crash    Fire Strike   Brutal Blast    D  100   True   False  False
magic
  Command 1st Ingredient 2nd Ingredient Type